In [1]:
! pip install jedi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00


In [2]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 109.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 1

In [1]:
import warnings

import pandas as pd

import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# Google Data
# train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
# eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)

# train_df = train_df.loc[train_df["label"] == "1"]
# eval_df = eval_df.loc[eval_df["label"] == "1"]

# train_df = train_df.rename(
#     columns={"sentence1": "input_text", "sentence2": "target_text"}
# )
# eval_df = eval_df.rename(
#     columns={"sentence1": "input_text", "sentence2": "target_text"}
# )

# train_df = train_df[["input_text", "target_text"]]
# eval_df = eval_df[["input_text", "target_text"]]

# train_df["prefix"] = "paraphrase"
# eval_df["prefix"] = "paraphrase"

# # MSRP Data
# train_df = pd.concat(
#     [
#         train_df,
#         load_data("data/msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )
# eval_df = pd.concat(
#     [
#         eval_df,
#         load_data("data/msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality"),
#     ]
# )

# # Quora Data

# # The Quora Dataset is not separated into train/test, so we do it manually the first time.
# df = load_data(
#     "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
# )
# q_train, q_test = train_test_split(df)

# q_train.to_csv("data/quora_train.tsv", sep="\t")
# q_test.to_csv("data/quora_test.tsv", sep="\t")

# # The code block above only needs to be run once.
# # After that, the two lines below are sufficient to load the Quora dataset.

# # q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
# # q_test = pd.read_csv("data/quora_test.tsv", sep="\t")

# train_df = pd.concat([train_df, q_train])
# eval_df = pd.concat([eval_df, q_test])


# MSRP Data
train_df = load_data("data/msr_paraphrase_train.txt", "#1 String", "#2 String", "Quality")
eval_df = load_data("data/msr_paraphrase_test.txt", "#1 String", "#2 String", "Quality")

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = 1
model_args.num_beam_groups = 0
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
)


# model = Seq2SeqModel(
#     encoder_decoder_type="bart",
#     encoder_decoder_name="facebook/bart-large",
#     args=model_args,
# )

model.train_model(train_df, eval_data=eval_df, output_dir="data/text_gen")

# to_predict = [
#     prefix + ": " + str(input_text)
#     for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
# ]
# truth = eval_df["target_text"].tolist()

# preds = model.predict(to_predict)

# # Saving the predictions if needed
# os.makedirs("predictions", exist_ok=True)

# with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
#     for i, text in enumerate(eval_df["input_text"].tolist()):
#         f.write(str(text) + "\n\n")

#         f.write("Truth:\n")
#         f.write(truth[i] + "\n\n")

#         f.write("Prediction:\n")
#         for pred in preds[i]:
#             f.write(str(pred) + "\n")
#         f.write(
#             "________________________________________________________________________________\n"
#         )



<ipython-input-1-7a8d3b999365>:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
b'Skipping line 1972: expected 5 fields, saw 6\nSkipping line 2890: expected 5 fields, saw 6\nSkipping line 3668: expected 5 fields, saw 6\nSkipping line 4054: expected 5 fields, saw 6\nSkipping line 4496: expected 5 fields, saw 6\n'
<ipython-input-1-7a8d3b999365>:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
b'Skipping line 64: expected 5 fields, saw 6\nSkipping line 424: expected 5 fields, saw 6\nSkipping line 528: expected 5 fields, saw 6\nSkipping line 1474: expected 5 fields, saw 6\nSkipping line 1702: expected 5 fields, saw 6\n'


          prefix                                         input_text  \
1     paraphrase  They​​ ha‌d‌ ​publish‍‍e‌d a‍n​‍‍ ad‌v‌e‌rtise...   
3     paraphrase  T​‌he‌ s​‌​toc‍k ro‌se ‌​$​‌​2.‌11, or‌ ‌about...   
4     paraphrase  Rev‍​e​‍‌‌nue i‌n‌ t​​​he first qua‌r‌ter‌ ‌of...   
6     paraphrase  T​he​​ ​D​V‍​​D-C‌CA then ‌a​p‌​‌‍‍‍pe​aled​‍​...   
8     paraphrase  He‌ ​sai‌d th​‌​​e‍ f‌oo​d‌s‌e‍‌rv‍ice​ ​‌‌‍‍​...   
...          ...                                                ...   
2450  paraphrase  ‍Att​a​c‌k‍‌ers ​detona​t​e‌d a​ sec‌ond​‍ ​ro...   
2453  paraphrase  Southw‍​e‍‍‌s‌t ‌s‌ai‍​d i‍t ‍r‍‍​e‍‍c‍​‍‌e‍nt...   
2455  paraphrase  ‍​​M​‍‌y ju​‍‍dgmen​​t‌‍‍ is‌​‌‌ ​​95 p‍ercen​...   
2456  paraphrase  F​o‍​l​​lowi‌ng the ‍​‌ATP's notifica‍t‌ion ​b...   
2458  paraphrase  Do‌‌‌t‍so​n‌‌‌​‌‍,‌ ‍21​,​ ‌‌adm‍​it‌te‍d t​o‍...   

                                            target_text  
1     O​‌n‌‌ ​June‍ ‌​10‌,‌​ the​ ship'‌s​​ ​o‌‍w​ne...  
3     PG&‍E‌ Corp. ​s​ha‍re‍‌​s

  0%|          | 0/1627 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

Running Epoch 0 of 2:   0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/642 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/642 [00:00<?, ?it/s]

(408,
 {'global_step': [204, 408],
  'eval_loss': [2.279702576723966, 2.1973031000657515],
  'train_loss': [2.4638588428497314, 2.2130720615386963]})

In [ ]:
#model.save_model("data/text_gen")

In [ ]:
#model.save("data/text_gen_1")

In [2]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)


model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)


#while True:
    #original = input("Enter text to paraphrase: ")
original = "The world's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected."
to_predict = [original]

preds = model.predict(to_predict)

print("---------------------------------------------------------")
print(original)

print()
print("Predictions >>>")
for pred in preds[0]:
    print(pred)

print("---------------------------------------------------------")
print()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
The world's two largest automakers said their U.S. sales declined more than predicted last month as a late summer sales frenzy caused more of an industry backlash than expected.

Predictions >>>
Sterd‌e,‍​​‍‌s‌‌, said:
Lufthansa and BMW t​h‌e‍s p‌​o‌‌r​‌​​l‍e​‍‍​ ‌​�
The world's two largest automakers‌ s‍​‌‍c‍on​t‍‌ed‍ last year's‌ t​o‍l ‍‍"‍s‌
---------------------------------------------------------

